# TIFF to MP4/GIF Conversion

In [27]:
import tifffile as tiff
import os
import numpy as np
import dask.array as da
import dask
from dask.delayed import delayed
from dask.diagnostics import ProgressBar
import imageio

# Set Dask to use multi-threading for better performance
dask.config.set(scheduler="threads")

def convert_tiff_to_video(tiff_path: str, img_name: list[str], img_shape: list[tuple], file_format: str = 'mp4', video_length: int = 100, fps: int = None, speed_factor: float = 1.0):
    """
    Converts a folder of TIFF images into a video animation (MP4 or GIF) using imageio.
    Converts each image to binary where all values > 0 are set to 255.
    
    Parameters:
        tiff_path (str): Base path to the folder containing TIFF images.
        img_name (list of str): List of folder names to process.
        img_shape (list of tuple): List of the dimensions of the images.
        file_format (str): Format of the output video ('mp4' or 'gif'). Defaults to 'mp4'.
        video_length (int): Desired length of the video in seconds (automatically calculates FPS). Only works if fps=None. Defaults to 100.
        fps (int): Frames per second for the output video. Overrides video_length. Defaults to None.
        speed_factor (float): Factor to speed up video by. Defaults to 1.0.
    """
    if len(img_name) != len(img_shape):
        raise ValueError(f"The number of image names and image shapes do not match")

    video_path = tiff_path.replace('tiff', 'video')
    os.makedirs(video_path, exist_ok=True)

    for name, shape in zip(img_name, img_shape):
        folder_path = os.path.join(tiff_path, name)
        save_path = os.path.join(video_path, name)
        os.makedirs(save_path, exist_ok=True)
        output_file = os.path.join(save_path, f"{name}.{file_format}")

        tiff_files = sorted(
            [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.tiff')]
        )

        if not tiff_files:
            print(f"Warning: No TIFF files found in {folder_path}. Skipping...")
            continue

        # Read images, convert to binary, and create a list of frames
        lazy_images = [delayed(lambda f: (tiff.imread(f) > 0).astype(np.uint8) * 255)(f) for f in tiff_files]
        image_stack = da.stack([da.from_delayed(img, shape=shape, dtype=np.uint8) for img in lazy_images])

        # Compute the image stack (parallelized)
        with ProgressBar():
            image_stack = image_stack.compute()

        num_frames = image_stack.shape[0]
        if fps is None:
            fps = max(1, round(num_frames / video_length))

        # Save frames directly into a video file (no napari display)
        output_file = os.path.join(save_path, f"{name}.{file_format}")
        with imageio.get_writer(output_file, mode='I', format='FFMPEG', fps=fps) as writer:
            for i in range(num_frames):
                writer.append_data(image_stack[i])

        print(f"Animation saved at: {output_file}")

In [28]:
# tiff_path = r'D:\Darren\Files\outputs\tiff\pretrained_gen35_zscore4'
# tiff_path = r'D:\Darren\Files\outputs\tiff\pretrained_clar35_zscore5'
tiff_path = r'D:\Darren\Files\outputs\tiff\pretrained_tab40_gen35_clar35_fold3'
# tiff_path = r'D:\Darren\Files\outputs\tiff\pretrained_tab40_gen35_clar35_foldsALL'

# convert_tiff_to_video(tiff_path, ['1_Microsphere'], [(1373, 1041)], 'mp4', video_length=45)
convert_tiff_to_video(tiff_path, ['2_Tablet'], [(747, 593)], 'mp4', video_length=45)
# convert_tiff_to_video(tiff_path, ['3_SprayDriedDispersion'], [(824, 824)], 'mp4', video_length=30)
convert_tiff_to_video(tiff_path, ['4_GenericD12'], [(954, 569)], 'mp4', video_length=30)
convert_tiff_to_video(tiff_path, ['5_ClaritinD12'], [(1004, 589)], 'mp4', video_length=30)
# convert_tiff_to_video(tiff_path, ['6_30pWTMCC'], [(1541, 1541)], 'mp4', video_length=30)

[########################################] | 100% Completed | 3.50 sms


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (747, 593) to (752, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Animation saved at: D:\Darren\Files\outputs\video\pretrained_tab40_gen35_clar35_fold3\2_Tablet\2_Tablet.mp4
[########################################] | 100% Completed | 3.19 sms


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (954, 569) to (960, 576) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Animation saved at: D:\Darren\Files\outputs\video\pretrained_tab40_gen35_clar35_fold3\4_GenericD12\4_GenericD12.mp4
[########################################] | 100% Completed | 3.45 sms


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1004, 589) to (1008, 592) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Animation saved at: D:\Darren\Files\outputs\video\pretrained_tab40_gen35_clar35_fold3\5_ClaritinD12\5_ClaritinD12.mp4
